In [1]:
import base64
import json
import requests


In [3]:
raw_bytes = open('/root/imgs/1.png', "rb").read()
raw_bytes[:10]

b'\x89PNG\r\n\x1a\n\x00\x00'

In [4]:
encoded_bytes = base64.b64encode(raw_bytes)
print(type(encoded_bytes))
encoded_bytes[:10]

<class 'bytes'>


b'iVBORw0KGg'

In [5]:

encoded_string = encoded_bytes.decode()
print(type(encoded_string))
encoded_string[:10]

<class 'str'>


'iVBORw0KGg'

In [2]:

def query(addr, filename):
    url = "http://%s/image-example/predict" % addr
    req_json = json.dumps({
        "input":
        base64.b64encode(open(filename, "rb").read()).decode() # bytes to unicode
    })
    headers = {'Content-type': 'application/json'}
    print(req_json)
    r = requests.post(url, headers=headers, data=req_json)
    print(r.json())

In [3]:
def image_size(imgs):
    """
    Input: 
    - imgs : (np.ndarray) of shape (n, d). n is the number of data in this batch
             d is the length of the bytes as numpy int8 array.  
    Output:
    - sizes : List[Tuple(int, int),...]
    """
    import base64
    import io
    import os
    import PIL.Image
    import tempfile
  
    num_imgs = len(imgs)
    
    sizes = []
    for i in range(num_imgs):
        # Create a temp file to write to
        tmp = tempfile.NamedTemporaryFile('wb', delete=False, suffix='.png')
        tmp.write(io.BytesIO(imgs[i]).getvalue())
        tmp.close()
        
        # Use PIL to read in the file and compute size
        size = PIL.Image.open(tmp.name, 'r').size
        
        # Remove the temp file
        os.unlink(tmp.name) 

        sizes.append(size)
    return sizes

In [4]:

from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as python_deployer

clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()


python_deployer.create_endpoint(
    clipper_conn=clipper_conn, 
    name="image-example", 
    input_type="bytes", 
    func=image_size, 
    pkgs_to_install=['pillow']
)

20-01-14:01:52:11 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-01-14:01:52:15 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmp2jgre5rd.yml
20-01-14:01:52:16 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running
20-01-14:01:52:16 INFO     [clipper_admin.py:236] [default-cluster] Application image-example was successfully registered
20-01-14:01:52:16 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpz9czzp7pclipper
20-01-14:01:52:16 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-01-14:01:52:16 INFO     [python.py:192] Python closure saved
20-01-14:01:52:16 INFO     [python.py:206] Using Python 3.6 base image
20-01-14:01:52:16 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpz9czzp7pclipper
20-01-14:01:52:17 INFO     [clipper_admin.py:539] [default-cluster] Step 1/3 : F

In [7]:

query(clipper_conn.get_query_addr(), '/root/imgs/2.png')

{"input": "..."}
{'query_id': 1, 'output': '(420, 764)', 'default': False}


In [11]:
def query_json(addr, filename, image_format):
    url = "http://%s/image-example-string/predict" % addr
    req_json = json.dumps({
        "input":
        json.dumps({
            'data': base64.b64encode(open(filename, "rb").read()).decode(),
            'format': image_format
        })
    })
    headers = {'Content-type': 'application/json'}
    r = requests.post(url, headers=headers, data=req_json)
    print(r.json())

In [12]:
def image_size_json(imgs):
    """
    Input: 
    - imgs : an array of strings 
    Output:
    - sizes : List[Tuple(int, int),...]
    """
    import base64
    import io
    import os
    import PIL.Image
    import tempfile
    import json
  
    num_imgs = len(imgs)
    sizes = []
    for i in range(num_imgs):
        # Deserialize the query
        data = json.loads(imgs[i])
        image_format = data['format']
        image_bytes = data['data'].encode()
        
        # Create a temp file to write to
        tmp = tempfile.NamedTemporaryFile('wb', delete=False, suffix='.{}'.format(image_format))
        tmp.write(io.BytesIO(base64.b64decode(image_bytes)).getvalue())
        tmp.close()
        
        # Use PIL to read in the file and compute size
        size = PIL.Image.open(tmp.name, 'r').size
        
        # Remove the temp file
        os.unlink(tmp.name) 

        sizes.append(size)
    return sizes

In [13]:

python_deployer.create_endpoint(
    clipper_conn=clipper_conn, 
    name="image-example-string", 
    input_type="strings", 
    func=image_size_json, 
    pkgs_to_install=['pillow']
)

20-01-09:04:25:19 INFO     [clipper_admin.py:236] [default-cluster] Application image-example-string was successfully registered
20-01-09:04:25:19 INFO     [deployer_utils.py:41] Saving function to /tmp/tmp21bhbyl1clipper
20-01-09:04:25:19 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-01-09:04:25:19 INFO     [python.py:192] Python closure saved
20-01-09:04:25:19 INFO     [python.py:206] Using Python 3.6 base image
20-01-09:04:25:19 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmp21bhbyl1clipper
20-01-09:04:25:20 INFO     [clipper_admin.py:539] [default-cluster] Step 1/3 : FROM clipper/python36-closure-container:0.4.1
20-01-09:04:25:20 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
20-01-09:04:25:20 INFO     [clipper_admin.py:539] [default-cluster] Step 2/3 : RUN apt-get -y install build-essential && pip install pillow
20-01-09:04:25:20 INFO     [clipper_admin.py:539] [default-

In [14]:
query_json(clipper_conn.get_query_addr(), '/root/imgs/3.jpg', 'jpg')

{'query_id': 2, 'output': '(260, 565)', 'default': False}


In [15]:
query_json(clipper_conn.get_query_addr(), '/root/imgs/2.png', 'png')

{'query_id': 3, 'output': '(420, 764)', 'default': False}


In [16]:

clipper_conn.stop_all()


20-01-09:04:34:14 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers
